# Factorización LU

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
M = 20 # M: Máximo valor de la distribución
A = M*np.random.rand(3,3)
A

In [3]:
def fact_lu(A):
    m, n = np.shape(A)
    if m != n:
        return None
    L = np.zeros((n, n))
    U = np.copy(A)
    for j in range(n-1):
        for i in range(j+1,n):
            L[i,j] = (U[i,j])/(U[j,j])
            U[i] = U[i] - L[i,j]*U[j]
    L = L + np.eye(n)
    return L, U

In [4]:
L, U = fact_lu(A)
L

In [5]:
U

In [6]:
A - L@U

In [7]:
np.linalg.norm(A - L@U)

# Factorización PLU

In [8]:
def swap_rows(A, i, j):
    temp = np.copy(A[i])
    A[i] = A[j]
    A[j] = temp

In [9]:
import numpy as np


def fact_plu(A):
    """
    Factoriza la matriz A en las matrices P, L y U utilizando la factorización PLU.
    
    Parameters:
    -----------
    A : np.ndarray
        Matriz a factorizar.
    
    Returns:
    --------
    P, L, U : np.ndarray
        Matrices de permutación (P), triangular inferior (L) y triangular superior (U).
    """
    m, n = np.shape(A)
    if m != n:
        raise ValueError("La matriz debe ser cuadrada")
    
    P = np.eye(n)  # Matriz de permutación
    L = np.zeros((n, n))  # Matriz triangular inferior
    U = np.copy(A)  # Copia de la matriz original
    
    for j in range(n - 1):
        # Pivoteo parcial
        if np.abs(U[j, j]) == 0:
            max_row = np.argmax(np.abs(U[j:, j])) + j
            if U[max_row, j] == 0:
                raise ValueError("La matriz es singular")
            swap_rows(U, j, max_row)
            swap_rows(P, j, max_row)
        
        for i in range(j + 1, n):
            L[i, j] = U[i, j] / U[j, j]
            U[i] -= L[i, j] * U[j]
    
    L += np.eye(n)  # Añadir la diagonal de unos a L
    return P, L, U

def forward_substitution(L, b):
    """
    Resuelve el sistema Ly = b, donde L es una matriz triangular inferior.
    
    Parameters:
    -----------
    L : np.ndarray
        Matriz triangular inferior.
    b : np.ndarray
        Vector independiente.
    
    Returns:
    --------
    y : np.ndarray
        Solución del sistema Ly = b.
    """
    n = len(b)
    y = np.zeros_like(b)
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    return y

def backward_substitution(U, y):
    """
    Resuelve el sistema Ux = y, donde U es una matriz triangular superior.
    
    Parameters:
    -----------
    U : np.ndarray
        Matriz triangular superior.
    y : np.ndarray
        Vector independiente.
    
    Returns:
    --------
    x : np.ndarray
        Solución del sistema Ux = y.
    """
    n = len(y)
    x = np.zeros_like(y)
    
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
    
    return x

def solve_plu(A, b):
    """
    Resuelve el sistema Ax = b usando la factorización PLU.
    
    Parameters:
    -----------
    A : np.ndarray
        Matriz de coeficientes A.
    b : np.ndarray
        Vector de términos independientes b.
    
    Returns:
    --------
    x : np.ndarray
        Solución del sistema Ax = b.
    """
    # Factorización PLU
    P, L, U = fact_plu(A)
    
    # Resolver Ly = Pb
    Pb = np.dot(P, b)
    y = forward_substitution(L, Pb)
    
    # Resolver Ux = y
    x = backward_substitution(U, y)
    
    return x


In [10]:
M = 20 # M: Máximo valor de la distribución
A = M*np.random.rand(3,3)
A

In [11]:
P, L, U = fact_plu(A)
P

In [12]:
L

In [13]:
U

In [14]:
P@A - L@U

In [15]:
np.linalg.norm(P@A - L@U)

In [16]:
errores_LU = []
enes = list(range(3,301,3))
for n in enes:
    M = 10 # M: Máximo valor de la distribución
    A = M*np.random.rand(n,n)
    L, U = fact_lu(A)
    errores_LU.append(np.linalg.norm(A - L@U))

In [17]:
errores_PLU = []
for n in enes:
    M = 10 # M: Máximo valor de la distribución
    A = M*np.random.rand(n,n)
    P, L, U = fact_plu(A)
    errores_PLU.append(np.linalg.norm(P@A - L@U))

In [18]:
plt.plot(enes, errores_LU, 'b--o', label='LU')
plt.plot(enes, errores_PLU, 'r--o', label='PLU')
plt.xlabel('Dimensión de la matriz (n)')
plt.ylabel('Error')
plt.title('LU vs PLU')
plt.grid(alpha=0.4)
plt.legend()

# Solución del SEL

In [19]:
def sel(A, b):
    L, U = fact_lu(A)
    n = np.size(A[0])
    y = np.zeros((n,1))
    y[0] = b[0]
    for k in range (1,n):
        y[k] = b[k]-L[k,0:k]@y[0:k]
    x = np.zeros((n,1))
    x[n-1] = y[n-1]/U[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (y[k] - U[k,k+1:n]@x[k+1:n,0])/U[k,k]
    return x, y, L, U

In [20]:
A = np.matrix([np.array([4.,5.,2.,-1.]),
               np.array([5.,8.,7.,6.]),
               np.array([3.,7.,-4.,-2.]),
               np.array([-1.,6.,-2.,5.])])
b = np.matrix([3.,2.,0.,1.]).T

In [21]:
x, y, L, U = sel(A, b)

In [22]:
x

In [23]:
y

In [24]:
L

In [25]:
U

In [26]:
A = np.matrix([np.array([1.,1.,0.,3.]),
               np.array([2.,1.,-1.,1.]),
               np.array([3.,-1.,-1.,2.]),
               np.array([-1.,2.,3.,-1.])])

b = np.matrix([8.,7.,14.,-7.]).T

x, y, L, U = sel(A, b)

In [27]:
print(x)
print(y)
print(L)
print(U)